In [1]:
from elasticsearch import Elasticsearch, helpers
import csv

In [2]:
elas = Elasticsearch(hosts="http://localhost:9200")


In [3]:
if not elas.ping():
    raise ValueError("Failed to connect to Elasticsearch!")

/tmp/ipykernel_9766/2353910976.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not elas.ping():


In [4]:
index_name = 'news_10'
if not elas.indices.exists(index=index_name):
    elas.indices.create(index=index_name, ignore=400)

/tmp/ipykernel_9766/570545090.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not elas.indices.exists(index=index_name):


In [5]:
# تابع generate_actions برای خواندن داده‌ها از فایل CSV و تولید مستند‌ها برای ارسال به Elasticsearch
def generate_actions():
    with open('news_202307170512.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for row in reader:
            # پیش‌پردازش مقادیر
            # حذف مقادیر نال
            for key, value in row.items():
                if value.strip() == '':
                    row[key] = None        
            # جداسازی مقادیر در ستون "tag" با کاراکتر |
            row['tags'] = row['tags'].split('|') if row.get('tags') else []
            yield {
                "_index": index_name,
                "_source": row
            }

# اضافه کردن داده‌ها به Elasticsearch با استفاده از helpers.bulk و پیش‌پردازش داده‌ها
helpers.bulk(elas, generate_actions())

/tmp/ipykernel_9766/2977199054.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(elas, generate_actions())


(90, [])

In [6]:
# تعداد سطرها و ستون‌های پس از پیش‌پردازش
with open('news_202307170512.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter='\t')
    rows = sum(1 for _ in reader)

with open('news_202307170512.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    header = next(reader)
    columns = len(header)

print("تعداد سطرها:", rows)
print("تعداد ستون‌ها:", columns)


تعداد سطرها: 90
تعداد ستون‌ها: 10
